# Semi-Automated Seagrass Classification Using Earth Engine Python API

This script classify seagrass beds in selected BOA images using ground-data to train the Support Vector Machine classifier. The outputs can be exported to EE Assets. All the training and validation matrices and accuracies can be saved as an Excel file in your working directory.<br/>


Script by: Luis Lizcano-Sandoval<br/>
College of Marine Sciences, University of South Florida<br/>
luislizcanos@usf.edu
Updated: 11/11/2021

<font size="4">**Workflow:**</font>

1. Import required images, collections, data, etc.
2. Mask clouds and land
3. Apply Depth-Invariant Index (generates band-ratios B1B2, B1B3, B2B3)
4. Sample bands: B1, B2, B3, B4, B/G
5. Train models and classify (CART, SVM and RF)
6. Get confusion matrices and accuracies
7. Export output to EE Assets (.tiff)
8. Save matrices in local computer (.xlxs)

In [1]:
## Run this cell to mount your Google Drive
import os, sys
from google.colab import drive
drive.mount('/content/drive')
nb_path = '/content/notebooks'
os.symlink('/content/drive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)  # or append(nb_path)

Mounted at /content/drive


In [2]:
## Authenticate your EE account
!earthengine authenticate

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=JbEmKG7ilCMmNsCrol_IzUUo6AfeXobYHGNqrokk-HY&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWjJY8iNUsv-ApG8nik0Z1yLG5dQAQGmLtMEBEAF23gg0qxl2smCXBk

Successfully saved authorization token.


In [3]:
## Install some libraries:
!pip install xlsxwriter

     |████████████████████████████████| 149 kB 8.6 MB/s 


In [4]:
## Clone github repo:
!git clone https://github.com/luislizcano/ee_seagrass_classification.git

Cloning into 'ee_seagrass_classification'...
remote: Enumerating objects: 88, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 88 (delta 41), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (88/88), done.


**Load required libraries:**

In [1]:
import os, sys
sys.path.insert(0,'/content/ee_seagrass_classification')
sys.path.append('/content/ee_seagrass_classification/bin/')
import ee
import pandas as pd
import xlsxwriter
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()),'bin'))
import datetime
# from functions import CloudScore6S,landMaskFunction,DII

ee.Initialize()

In [2]:
## Verify you loaded the EE module correctly:
from functions import CloudScore6S,landMaskFunction,DII
print('EE version: ',ee.__version__)

EE version:  0.1.290


**Insert the ID of the image of interest and folder from the personal Assets:**

In [7]:
## Select set of images belonging to a same satellite sensor and year.
## cienaga: 19PFM; PtoAzul: 19PGM; PNM: 19PEN
## Sentinel 2
imageList = [
'20200522T150731_20200522T150726_T19PEN',
'20200527T150719_20200527T150721_T19PEN'
]

## Landsat list
# 2019
# imageList = [
# 'LE07_017040_20190223',
# 'LE07_017040_20191005']

# 2000
# imageList = [
# 'LE07_017040_20000914',
# 'LT05_017040_20000602']

# 1990
# imageList = [
# 'LT05_017041_19990904',
# 'LT05_017041_19991107',
# 'LT05_017041_19990107'
# ]

# for i in range(len(imageList)):
#     print(imageList[i])

**Some metadata:**

In [4]:
## Define the source of your image, if from your EE Assets or EE Collections:
# imageSource = 'assets'  ## BOA imagery from EE Assets
imageSource = 'ee'      ## BOA imagery from EE Collections

## Define the type of satellite imagery
satellite = 'Sentinel2'
# satellite = 'Landsat8'
# satellite = 'Landsat7'
# satellite = 'Landsat5'

## Some settings:
regionName = 'cienaga' ## Region to classify [from ´regions´ collection below]
#boaFolder = 'FL_20' ## Name of folder with BOA images in assets.
exportFolder = 'Cienaga_20' ## Name of EE folder to save the final output
dataFolder = 'Ground-points-19' ## EE Folder with Ground-truth points
smoothStr = '_raw_' # Smooth classified pixels or not? options: '_smooth_' or '_raw_'

**Import data:**

In [8]:
###########################    IMPORT TRAINING DATA    #########################

# Sandy areas
sand_areas = ee.FeatureCollection("users/anacaroperalta/datos/sand_poly_2020")

# Ground-Points
groundPoints = ee.FeatureCollection("users/anacaroperalta/datos/puntos_clasificacion2020")
#groundPoints = ee.FeatureCollection("users/lizcanosandoval/ground-points/Turkey_2020")

## Landmask
#land = ee.FeatureCollection("users/lizcanosandoval/Florida_10m") ##Created from NDWI using Sentinel-2 imagery
mask1 = ee.Image('users/anacaroperalta/datos/waterMask_PNM_S')
mask2 = ee.Image('users/anacaroperalta/datos/waterMask_PtoAzul_19PGM')
mask3 = ee.Image('users/anacaroperalta/datos/waterMask_cienaga_202019PFM')

land = ee.ImageCollection([mask1,mask2,mask3]) ##Created from NDWI using Sentinel-2 imagery

## Region polygons: (cienaga,PtoAzul, PNM)
regions = ee.FeatureCollection("users/anacaroperalta/datos/poligonosinteres")

print('Collections loaded')

Collections loaded


**Start classification loop:**

In [9]:
%%time
print('Initiating...')

## Initiate loop:
for i in range(len(imageList)):
    imageID = imageList[i]
    
    print('Preparing image '+imageID)
    
    #############################   Prepare image metadata  ##################################

    ## If the image source is your asset, then define the folder where the satellite image is:
    if 'assets'== imageSource:
        ## Load BOA image from assets:
        if 'Sentinel' in satellite:
            imageTarget = ee.Image("users/lizcanosandoval/BOA/Sentinel/"+boaFolder+'/'+imageID)
        elif 'Landsat' in satellite:
            imageTarget = ee.Image("users/lizcanosandoval/BOA/Landsat/"+boaFolder+'/'+imageID)

    ## If the image source is an EE collection, then define the satellite collection:
    if 'ee'== imageSource:
        ## Load BOA image collection from assets:
        if 'Sentinel' in satellite:
            image = ee.Image("COPERNICUS/S2_SR/"+imageID)
        elif 'Landsat8' == satellite:
            image = ee.Image("LANDSAT/LC08/C01/T1_SR/"+imageID)
        elif 'Landsat7' == satellite:
            image = ee.Image("LANDSAT/LE07/C01/T1_SR/"+imageID)
        elif 'Landsat5' == satellite:
            image = ee.Image("LANDSAT/LT05/C01/T1_SR/"+imageID)

    ## Get image metadata:
    if 'assets'== imageSource:
        imageSat = imageTarget.get('satellite').getInfo() #Image satellite
        imageTile = imageTarget.get('tile_id').getInfo() #Image tile id
        imageDate = imageTarget.get('date').getInfo() #Image date
        imageGeometry = imageTarget.geometry() #Tile geometry.

    if 'ee'== imageSource:
        if 'Sentinel' in satellite:
            imageTarget = image.divide(10000).set(image.toDictionary(image.propertyNames()))
            imageSat = imageTarget.get('SPACECRAFT_NAME').getInfo() #Image satellite
            imageTile = imageTarget.get('MGRS_TILE').getInfo() #Image tile id
            ee_date = imageTarget.get('GENERATION_TIME').getInfo()
            imageDate = str(datetime.datetime.utcfromtimestamp(ee_date/1000.0)) #Image date
            imageGeometry = imageTarget.geometry() #Tile geometry.
        elif 'Landsat8' == satellite:
            imageTarget = image.divide(10000).set(image.toDictionary(image.propertyNames()))
            #imageSat = imageTarget.get('SATELLITE').getInfo() #Image satellite
            imageSat = satellite
            imageTile = str(imageTarget.get('WRS_PATH').getInfo())+str(imageTarget.get('WRS_ROW').getInfo()) #Image tile id
            imageDate = imageTarget.get('SENSING_TIME').getInfo()
            imageGeometry = imageTarget.geometry() #Tile geometry.

    if 'Sentinel' in imageSat:
        imageScale = 10 # Sentinel resolution
    else:
        imageScale = 30 # Landsat resolution

    
    ###########################    CLOUD MASK    #########################

    ## Recommended Threshold values for
    ## *Sentinel: 2
    ## *Landsat: 5
    if 'Sentinel' in imageSat:
        threshold = 5
    else:
        threshold = 5

    ## Apply cloud mask
    cloudMask = CloudScore6S(imageSat, imageTarget, threshold)


    ###########################    LAND MASK    #########################

    ## Apply land mask
    #landMask = landMaskFunction(cloudMask, land) ## Use if Land is a featureCollection
    landMask = cloudMask.mask(land.mosaic()) ## Use if Land is an imageCollection


    ###########################    BATHYMETRY MASK    #########################

    ## Apply bathymetry mask
    bathyMask = landMask#.clip(bathyVector) ##Using the NOAA dataset: bathyVector


    ###########################    TURBIDITY MASK    #########################

    ## Turbidity masking is based on the red band reflectances. Based on own observations, values higher than 0.02-0.03 
    ## indicates turbid waters, but sometimes may indicate shallow seagrass banks. So the below algorithm try to separate 
    ## shallow seagrass from turbidity.

    ## Set parameter values
    if 'Sentinel' in imageSat:
        red_band = 'B4' #Red seems work better in this case than B5.
        red_thr_inf = 0.025
        red_thr_sup = 0.2
        green_band = 'B3'
        green_thr = 0.15
        blue_band = 'B2'
        blue_thr = 0.11
    elif 'Landsat8' in imageSat:
        red_band = 'B4'
        red_thr_inf = 0.025
        red_thr_sup = 0.2
        green_band = 'B3'
        green_thr = 0.15
        blue_band = 'B2'
        blue_thr = 0.11
    else:
        red_band = 'B3'
        red_thr_inf = 0.03 #Landsat5/7 are less sensitive
        red_thr_sup = 0.2
        green_band = 'B2'
        green_thr = 0.15
        blue_band = 'B1'
        blue_thr = 0.11

    ## Select the red band
    selectRedBand = bathyMask.select('B4')

    ## Identify turbid areas first (pixel values higher than the threshold)
    turbidMask = selectRedBand.gt(red_thr_inf)

    ## Apply thresholds on red, green and blue bands
    maskRed = selectRedBand.gt(red_thr_inf).And(selectRedBand.lt(red_thr_sup))
    imageMaskRed = bathyMask.mask(maskRed)
    maskGreen = imageMaskRed.select(green_band).lt(green_thr)
    imageMaskGreen = imageMaskRed.mask(maskGreen)
    maskBlue = imageMaskGreen.select(blue_band).lt(blue_thr) ##Shallow seagrass

    ## Final mask (excluding seagrass/including turbid water)
    turbidImage = bathyMask.mask(turbidMask) ## Turbidity
    seagrassImage = bathyMask.mask(maskBlue).mask().Not() ## Shallow seagrass (inverse mask)
    excludeSeagrass = turbidImage.updateMask(seagrassImage) ## Turbidity minus shallow seagrass
    finalMaskImage = bathyMask.mask(excludeSeagrass)
    finalMask = finalMaskImage.mask().Not()

    ## Final Image
    finalImage = bathyMask.updateMask(finalMask)
    print('   Image masked...')

    ###########################    WATER COLUMN CORRECTION    #########################    

    ## Filter sand polygons by tile/area:
    #sand = ee.FeatureCollection(sand_areas).flatten().filterBounds(imageGeometry)
    sand = ee.FeatureCollection(sand_areas).filterBounds(imageGeometry)

    ## Run the Depth-Invariant Index Function
    imageDII = DII(finalImage, imageScale, sand)

    print('   Depth-Invariant index applied...')

    ###########################    SAMPLING    #########################
    # Classes are:

    # 0: Softbottom
    # 1: Hardbottom
    # 2: Seagrass
    # 3: Sparse seagrass //if available

    ## Filter ground points by tile geometry and display classes
    filterPoints = ee.FeatureCollection(groundPoints).filterBounds(imageGeometry)

    ## Select bands to sample. The B/G band is B2B3 in Sentinel-2 and Landsat-8, and B1B2 for Landsat-7/5
    if 'Sentinel' in imageSat or 'Landsat8' in imageSat:
        bandsClass = ['B1','B2', 'B3', 'B4','B2B3']
        bg = ['B2B3']
    else:
        bandsClass = ['B1','B2', 'B3', 'B1B2']
        bg = ['B1B2']

    ## Add bands of interest to sample training points:
    imageClassify = finalImage.addBands(imageDII.select(bg)).select(bandsClass)



    ###########################    APPLY SMOOTHER    #########################
    ## Define a boxcar or low-pass kernel (Used if want to smooth the image)
    smooth = ee.Kernel.euclidean(**{
        'radius': 1, 
        'units': 'pixels', 
        'normalize': True
    })
    
    ## Apply smoother if set:
    if 'smooth' in smoothStr:
        imageClassify = imageClassify.convolve(smooth)
    

    ###########################    GET TRAINING AND VALIDATION DATA    #########################

    ## Sample multi-spectral data using all ground points.
    samplingData = imageClassify.sampleRegions(**{
        'collection': filterPoints,
        'properties': ['class'],
        'scale': imageScale})

    ## Add random numbers to each feature (from 0 to 1).
    randomData = samplingData.randomColumn("random",0)

    ## Split ground data in training (~70%) and validation (~30%) points
    trainingData = randomData.filter(ee.Filter.lt("random",0.7))
    validationData = randomData.filter(ee.Filter.gte("random", 0.7))



    ###########################    TRAIN MODELS AND CLASSIFY    #########################
    print('   Training models and classifying...')

    ## Train SVM classifier
    SVM = ee.Classifier.libsvm(**{
       'kernelType': 'RBF',
       'gamma': 100,
       'cost': 100
    })
    trainSVM = SVM.train(**{
       'features': trainingData,
       'classProperty': 'class',
       'inputProperties': bandsClass
    })


    #### Classify the image using the trained classifier
    classifiedSVM = imageClassify.classify(trainSVM)
    
    #########################  Clean classified image  ###########################
    # seagrass_mask = ee.Image("users/lizcanosandoval/Seagrass/SeagrassMask_FL_100m")
    # classifiedSVM = classifiedSVM.updateMask(seagrass_mask) #For raster
    aoi = regions.filter(ee.Filter.eq('name',regionName))
    classifiedSVM = classifiedSVM.clip(aoi)


    ###########################    TRAINING ACCURACIES    #########################
    print('   Getting accuracies...')
    ## Get a confusion matrix representing resubstitution accuracy.
    ## {Resubstitution error is the error of a model on the training data.}
    ## Axis 0 (first level) of the matrix correspond to the input classes (columns), 
    ## and axis 1 (second level) to the output classes (rows).
    matrixTrainingSVM = trainSVM.confusionMatrix()


    ###########################    VALIDATION ACCURACIES    #########################

    ## Calculate accuracy using validation data
    ## Classify the image using the trained classifier
    validationSVM = validationData.classify(trainSVM)

    ## Get a confusion matrix representing expected accuracy (Using validation points - 30%), where:
    #  0: Softbottom
    #  1: Hardbottom
    #  2: Dense Seagrass
    #  3: Spare Seagrass

    ## Axis 0 (the rows) of the matrix correspond to the actual values, 
    ## and Axis 1 (the columns) to the predicted values.
    errorMx = {'actual': 'class', 'predicted': 'classification'}
    errorMatrixSVM = validationSVM.errorMatrix(**errorMx)


    ###########################    USER/PRODUCER ACCURACIES    #########################

    ## Estimate user and producer accuracies
    producerAccuracySVM = errorMatrixSVM.producersAccuracy()

    # USER
    userAccuracySVM = errorMatrixSVM.consumersAccuracy()


    ###########################    KAPPA COEFFICIENTS    #########################

    # The Kappa Coefficient is generated from a statistical test to evaluate the accuracy 
    # of a classification. Kappa essentially evaluate how well the classification performed 
    # as compared to just randomly assigning values, i.e. did the classification do better 
    # than random. The Kappa Coefficient can range from -1 to 1. A value of 0 indicated that 
    # the classification is no better than a random classification. A negative number 
    # indicates the classification is significantly worse than random. A value close to 1 
    # indicates that the classification is significantly better than random.
    kappaSVM = errorMatrixSVM.kappa()



    ###########################    EXPORT CLASSIFIED IMAGES    #########################

    # Set the scale properly
    scale = []
    sat = []
    method = ['SVM']
    classifiedCollection = ee.ImageCollection([classifiedSVM])
    classifiedList = classifiedCollection.toList(classifiedCollection.size())
    classifiedSize = classifiedList.size().getInfo()
    print('   Exporting classified images to EE Assets...')

    for i in range(classifiedSize):

        # Rename satellite
        if 'Sentinel' in imageSat:
            sat = 'Sentinel'
        else:
            sat = 'Landsat'

        ## Select image
        image = ee.Image(classifiedList.get(i))

        # set some properties for export
        output = image.set({'satellite': imageSat,
                       'tile_id': str(imageTile),
                       'file_id': imageID,                                               
                       'date': imageDate,
                       'year': imageDate[0:4],
                       'classifier': method[i],
                       'generator': 'Lizcano-Sandoval',
                            })

        # define YOUR assetID. (This do not create folders, you need to create them manually)
        assetID = 'users/lizcanosandoval/Seagrass/'+sat+'/Venezuela/'+exportFolder+'/' ##This goes to an ImageCollection folder
        fileName = imageID+smoothStr+ method[i]
        path = assetID + fileName

        ## Batch Export to Assets
        ee.batch.Export.image.toAsset(\
            image = ee.Image(output),                                                    
            description = method[i] +smoothStr+ imageID,
            assetId = path,
            region = imageGeometry.buffer(10),                                      
            maxPixels = 1e13,
            scale = imageScale).start()
        print('   Classified Image '+str(i+1)+': '+imageID +smoothStr+ method[i]+' submitted...')
    print('   Classified images submitted!')



    ###########################    SAVE MATRICES TO WORKING DIRECTORY    #########################
    print('   Saving matrices to working directory...')
    # Extract values from each matrix
    SVM_trainingMatrix = matrixTrainingSVM.array().getInfo()
    SVM_trainingAccuracy = matrixTrainingSVM.accuracy().getInfo()
    SVM_errorMatrix = errorMatrixSVM.array().getInfo()
    SVM_errorAccuracy = errorMatrixSVM.accuracy().getInfo()
    SVM_producerAccuracy = producerAccuracySVM.getInfo()
    SVM_userAccuracy = userAccuracySVM.getInfo()
    SVM_kappa = kappaSVM.getInfo()


    ## Convert matrices to pandas dataframes:
    #Training Matrices
    rowIndex = {0:'Sb', 1:'Hb', 2:'Dn', 3:'Sp'}
    TM_SVM = pd.DataFrame(SVM_trainingMatrix).rename(columns=rowIndex, index=rowIndex)
    TM_concat = pd.concat([TM_SVM], keys=['SVM'])

    #Training Accuracies
    TA_SVM = pd.Series(SVM_trainingAccuracy)
    TA_concat = pd.DataFrame(pd.concat([TA_SVM],ignore_index=True), columns=(['Tr_Accuracy']))\
                    .rename({0:'SVM'})

    #Validation-Error Matrices
    VM_SVM = pd.DataFrame(SVM_errorMatrix).rename(columns=rowIndex, index=rowIndex)
    VM_concat = pd.concat([VM_SVM], keys=['SVM'])

    #Validation Accuracies
    VA_SVM = pd.Series(SVM_errorAccuracy)
    VA_concat = pd.DataFrame(pd.concat([VA_SVM],ignore_index=True), columns=(['Va_Accuracy']))\
                    .rename({0:'SVM'})

    #Producer-User Accuracies
    ## Create a pandas dataframe with producer and user accuracies:
    dfPA_SVM = pd.DataFrame(producerAccuracySVM.getInfo(), columns=['Producer'])
    dfUA_SVM = pd.DataFrame(userAccuracySVM.getInfo()).transpose()

    PU_SVM = pd.concat([dfPA_SVM, dfUA_SVM.rename(columns={0:'User'})], axis=1).rename(index=rowIndex)
    PU_concat = pd.concat([PU_SVM], keys=['SVM'])

    # Kappa coefficients
    Kp_SVM = pd.Series(SVM_kappa)
    Kp_concat = pd.DataFrame(pd.concat([Kp_SVM],ignore_index=True), columns=(['Kappa']))\
                    .rename({0:'SVM'})

    # Extract the number of training and validation points per class:
    trainingInfo = trainingData.aggregate_histogram('class').getInfo()
    validationInfo = validationData.aggregate_histogram('class').getInfo()

    traSeries = pd.Series(trainingInfo)
    valSeries = pd.Series(validationInfo)

    Points_concat = pd.DataFrame(pd.concat([traSeries, valSeries],ignore_index=True,axis=1))\
                    .rename(columns={0:'TraPoints',1:'ValPoints'}).rename({'0':'Sb','1':'Hb','2':'Dn'},axis='index')
    
    # Organize each matrix in separate excel sheets
    excelName = 'Mrx'+ smoothStr + imageID +'.xlsx'
    excel = pd.ExcelWriter(excelName, engine='xlsxwriter')

    Points_concat.to_excel(excel, sheet_name='Points', index=True, startrow=0)
    TM_concat.to_excel(excel, sheet_name='TrMrx', index=True, startrow=0)
    TA_concat.to_excel(excel, sheet_name='TrAcc', index=True, startrow=0)
    VM_concat.to_excel(excel, sheet_name='VaMrx', index=True, startrow=0)
    VA_concat.to_excel(excel, sheet_name='VaAcc', index=True, startrow=0)
    PU_concat.to_excel(excel, sheet_name='PU-Mrx', index=True, startrow=0)
    Kp_concat.to_excel(excel, sheet_name='Kappa', index=True, startrow=0)

    # Save matrices as .xlsx file:
    excel.save()
    print('   Saved Matrices of '+imageID)

print('ALL IMAGES HAVE BEEN CLASSIFIED!')

Initiating...
Preparing image 20200522T150731_20200522T150726_T19PEN
   Image masked...
   Depth-Invariant index applied...
   Training models and classifying...
   Getting accuracies...


EEException: ignored